Author: [Ahmed Sobhi](ahmedyousrisobhi@gmail.com)

Creation_data: 24th Jul 2023

Objective: Data Cleaning

## Importing Libararies and Packages

In [13]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

from tqdm import tqdm

from dataprep.eda import create_report

# Used to get convert RGB values into colours
import webcolors

## Required Functions Definision

In [22]:
def calculate_discount(data:pd.DataFrame, target_col:str)->pd.DataFrame:
    """
        Used to calculate discount feature from ratio column.

        PARAMETERS
            data: pandas dataframe, input dataframe.
            target_col: str, target ratio column.
        
        OUTPUT
            pandas DataFrame with extracted features
    """

    # Make a copy of input dataset
    df_copy = data.copy()

    df_copy['discount'] = 1- df_copy['ratio']

    # Drop unnessary column
    df_copy.drop('ratio', axis=1, inplace=True)

    return df_copy
    
def closest_colour(requested_colour:tuple)->str:
    """
        Used to calculate the closest color based on RGB values.

        PARAMETERS
            requested_colour: tuple, values of RGB.
        
        OUTPUT
            str, the closest color name
    """
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]


def get_colour_name(requested_colour):
    """
        Used to calculate the actual and closest color based on RGB values.

        PARAMETERS
            requested_colour: tuple, values of RGB.
        
        OUTPUT
            list of strings, the actual and closest color name
    """
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name


def extract_closest_color(data:pd.DataFrame)->pd.DataFrame:
    """
        Used to extract closest color names based on RGB values.

        PARAMETERS
            data: pandas dataframe, input dataframe.
        
        OUTPUT
            pandas DataFrame with extracted colors
    """
    
    # Create a copy of color name
    df_copy = data.copy()

    # Define main/secondary colours
    main_color_cols = ['rgb_r_main_col', 'rgb_g_main_col', 'rgb_b_main_col'] 
    sec_color_cols = ['rgb_r_sec_col', 'rgb_g_sec_col', 'rgb_b_sec_col']

    # Calculate the closest color
    df_copy['main_closest_color'] = (
        df_copy[main_color_cols].apply(lambda row: tuple(row), axis=1)
        .apply(get_colour_name)
        .apply(lambda row: row[1])
    )

    df_copy['sec_closest_color'] = (
        df_copy[sec_color_cols].apply(lambda row: tuple(row), axis=1)
        .apply(get_colour_name)
        .apply(lambda row: row[1])
    )

    # Drop unnessary columns
    df_copy.drop(main_color_cols, axis=1, inplace=True)
    df_copy.drop(sec_color_cols, axis=1, inplace=True)

    return df_copy


def extract_time_based_features(data:pd.DataFrame, ts_col:str)->pd.DataFrame:
    """
        Used to extract time based features from time series column.

        PARAMETERS
            data: pandas dataframe, input dataframe.
            ts_col: str, target time series column.
        
        OUTPUT
            pandas DataFrame with extracted features
    """
    
    # make a copy of input dataframe
    df_copy = data.copy()

    # Extract time based features
    df_copy['year'] = df_copy[ts_col].dt.year
    df_copy['month'] = df_copy[ts_col].dt.month
    df_copy['day'] = df_copy[ts_col].dt.day
    df_copy['dayofweek'] = df_copy[ts_col].dt.day_of_week

    # drop timeseries column
    df_copy.drop(ts_col, axis=1, inplace=True)

    return df_copy

## Loading Dataset

In [15]:
df = (
    pd.read_csv('../data/intermid/full_gen_data.csv')
    .assign(
        retailweek = lambda x: pd.to_datetime(x.retailweek),
        customer_id = lambda x: x.customer_id.astype('int')
    )
)

df.head()

,country,article_id_1,sales,regular_price,current_price,ratio,retailweek,promo_media_ads,promo_store_event,customer_id,...,style,sizes,gender,rgb_r_main_col,rgb_g_main_col,rgb_b_main_col,rgb_r_sec_col,rgb_g_sec_col,rgb_b_sec_col,label
0,Austria,IR3275,5,26.95,22.95,0.851577,2015-07-26,0,0,4001,...,slim,"xxs,xs,s,m,l,xl,xxl",women,205,104,57,255,187,255,0
1,Austria,IR3275,5,26.95,22.95,0.851577,2015-07-26,0,0,4001,...,regular,"xxs,xs,s,m,l,xl,xxl",women,188,238,104,255,187,255,0
2,Austria,IR3275,5,26.95,22.95,0.851577,2015-07-26,0,0,4001,...,regular,"xxs,xs,s,m,l,xl,xxl",women,205,173,0,255,187,255,0
3,Austria,IR3275,5,26.95,22.95,0.851577,2015-07-26,0,0,4001,...,regular,"xxs,xs,s,m,l,xl,xxl",kids,205,140,149,164,211,238,1
4,Austria,IR3275,5,26.95,22.95,0.851577,2015-07-26,0,0,4001,...,regular,"xxs,xs,s,m,l,xl,xxl",women,138,43,226,164,211,238,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86520 entries, 0 to 86519
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   country            86520 non-null  object        
 1   article_id_1       86520 non-null  object        
 2   sales              86520 non-null  int64         
 3   regular_price      86520 non-null  float64       
 4   current_price      86520 non-null  float64       
 5   ratio              86520 non-null  float64       
 6   retailweek         86520 non-null  datetime64[ns]
 7   promo_media_ads    86520 non-null  int64         
 8   promo_store_event  86520 non-null  int64         
 9   customer_id        86520 non-null  int64         
 10  article_id_2       86520 non-null  object        
 11  productgroup       86520 non-null  object        
 12  category           86520 non-null  object        
 13  cost_article_2     86520 non-null  float64       
 14  style 

## Discount feature

In [ ]:
df = calculate_discount(df, 'ratio')

## Color Feature
- The RGB values for the primary and secondary articles in the dataset represent the intensity of the red, green, and blue primaries of the respective colors.

In [17]:
# Create a closest color feature
df = extract_closest_color(df)

## Time Based Features

In [23]:
df = extract_time_based_features(df, 'retailweek')